# Manipuler des données MARC avec la bibliothèque Python Pymarc

Objectif de l'atelier : être en mesur de lire un fichier MARC avec Python et d'en extraire des données.

Ce notebook est compatible avec la version 5.0.0 de Pymarc.

Pour la documentation, voir https://pymarc.readthedocs.io/en/latest/

## Quelques rappels sur le langage Python

### Structures de données

#### Listes

In [ ]:
# création d'une liste vide :
ma_liste = []
ma_liste

In [ ]:
# création d'une liste avec deux éléments :
ma_liste = ['tata', 'toto']
ma_liste

In [ ]:
# ajout d'un élément
ma_liste.append('titi')
ma_liste

In [ ]:
# accès à un élément de la liste (le 2e par exemple)
ma_liste[1]

#### Dictionnaires

In [ ]:
# création d'un dictionnaire vide
mon_dict = {}
mon_dict

In [ ]:
# création d'un dictionnaire avec deux paires clé/valeur :
mon_dict = {'tata': 'chien', 'toto': 'chat'}
mon_dict

In [ ]:
# ajout d'une paire clé/valeur :
mon_dict['titi'] = 'canari'
mon_dict

In [ ]:
# accès à une valeur via une clé
mon_dict['tata']

### Boucle "for"

In [ ]:
# elle permet une itération sur une liste par exemple :
# NB : attention à l'indentation
for a in ma_liste:
    print(a)

### Condition : "if"

In [ ]:
# on vérifie qu'une condition est vraie
a = 'tata'
if a in ma_liste:
    print(f"{a} est dans ma liste.")
else:
    print(f"{a} n'est pas dans ma liste.")

## Premiers pas avec Pymarc : lecture d'un fichier de notices

In [ ]:
# installation de Pymarc
# attention : cette étape ne vaut que pour les notebooks sous Google Colab
!pip install pymarc

In [ ]:
# import des données
!wget https://raw.githubusercontent.com/ragbx/pymarc-atelier-tb/main/data/notices.mrc

In [13]:
marc_file = 'notices.mrc'

In [14]:
# on charge la classe MARCReader
from pymarc import MARCReader

In [ ]:
# affichage des notices en format texte
with open(marc_file, 'rb') as fh:
    reader = MARCReader(fh, to_unicode=True, force_utf8=True)
    for record in reader:
        print(record)

In [ ]:
# petite variante : on compte le nombre de notices en même temps
i = 0
with open(marc_file, 'rb') as fh:
    reader = MARCReader(fh, to_unicode=True, force_utf8=True)
    for record in reader:
        i += 1
        print(record)

In [ ]:
print(f"Nb de notices : {i}")

## Extraire les données de champs et sous-champs

### Comprendre le focntionnement de Pymarc

In [ ]:
# dans un objet MARCReader, j'ai une ou plusieurs notices, soit un ou plusieurs objets "record"
records = []
with open(marc_file, 'rb') as fh:
    reader = MARCReader(fh, to_unicode=True, force_utf8=True)
    for record in reader:
        records.append(record)
# on récupère uniquement la 17e notice du fichier (parce que !)
record = records[16]
print(record)

In [ ]:
# dans un objet record, on a un label ('leader')...
record.leader

In [ ]:
# ... et des champs ('fields')
fields = record.fields
for field in fields:
    print(field)

In [ ]:
# dans les objets "field", on a des sous-champs ('subfields')
for subfield in field.subfields :
    print(subfield)

In [ ]:
# ... sous-champs auquel on peut aussi accéder de la sorte :
# dans les objets "field", on a des sous-champs ('subfields')
for subfield in field.subfields :
    print(f"${subfield.code} : {subfield.value}")

### Sélectionner des champs, sous-champs

#### Exemple : extraire un titre

##### Méthode get_fields()

In [ ]:
# si je veux selectionner un champ ('field') selon une zone ('tag') unimarc :
fields = record.get_fields('200')
for field in fields:
    print(field)

##### Méthode get_subfields()

In [ ]:
# si je veux selectionner un souschamp ('field') selon une zone ('tag') unimarc :
fields = record.get_fields('200')
for field in fields:
    subfields_a = field.get_subfields('a')
    print(subfields_a)
    subfields_e = field.get_subfields('e')
    print(subfields_e)

In [ ]:
fields = record.get_fields('200')
for field in fields:
    values = []
    for s in ['a', 'e']:
        for v in field.get_subfields(s):
            values.append(v)
value = " | ".join(values)
print(value)

#### Exemple 2 : sujets

In [ ]:
fields = record.get_fields('606')
for field in fields:
    values = []
    for s in ['a', 'x', 'y']:
        for v in field.get_subfields(s):
            values.append(v)
    value = " | ".join(values)
    print(value)

##### un petit problème....
get_subfields() ne prend pas les champs dans l'ordre de leur apparition

In [ ]:
fields = record.get_fields('606')
for field in fields:
    values = []
    for s in ['3', 'a', 'x', 'y']:
        for v in field.get_subfields(s):
            values.append(v)
    value = " | ".join(values)
    print(value)

##### une solution...
passer directement par les attributs code et value de subfield

In [ ]:
fields = record.get_fields('606')
for field in fields:
    values = []
    if hasattr(field, "subfields"):
        for subfield in field.subfields:
            if subfield.code in ['3', 'a', 'x', 'y']:
                values.append(subfield.value)
    value = " | ".join(values)
    print(value)

#### Construire une fonction "couteau suisse"
Syntaxe un peu pénible.
Le plus simple est de se construire une fonction qui à part d'étiquettes de champs et d'un ou plusieurs sous champs extrait les valeurs

In [29]:
def get_marc_values(record, tags):
    result = []
    for tag in tags:
        # cas du label
        if tag == 'LDR':
            result.append(record.leader)
        else:
            fields = record.get_fields(tag[:3])
            for field in fields:
                field_value = []
                # cas du controlfield
                if tag[:2] == '00':
                    field_value.append(field.data)
                # cas du datafield
                else:
                    if hasattr(field, "subfields"):
                        for subfield in field.subfields:
                            if subfield.code in tag[3:]:
                                field_value.append(subfield.value)
                result.append(" ".join(field_value))
    return " ; ".join(result)


In [ ]:
#exemple d'utilisation, en reprenant le cas précédent
get_marc_values(record, ['606axy'])


## Transformer un fichier marc en tableau excel

### On crée une fonction pour chaque colonne souhaitée

In [31]:
def get_type_notice(record):
    result = get_marc_values(record, ["LDR"])
    result = result[6]
    type_notice_codes = {
        "a": "texte",
        "b": "manuscrit",
        "c": "partition",
        "d": "partition manuscrite",
        "e": "carte",
        "f": "carte manuscrite",
        "g": "video",
        "i": "son - non musical",
        "j": "son - musique",
        "k": "image, dessin",
        "l": "ressource électronique",
        "m": "multimedia",
        "r": "objet"
    }
    if result in type_notice_codes.keys():
        result = type_notice_codes[result]
    return result

def get_niveau_bib(record):
    result = get_marc_values(record, ["LDR"])
    result = result[7]
    niveau_bib_codes = {
        "a": "analytique",
        "i": "ressource intégratrice",
        "m": "monographie",
        "s": "périodique",
        "c": "collection"
    }
    if result in niveau_bib_codes.keys():
        result = niveau_bib_codes[result]
    return result

def get_record_id(record):
    result = get_marc_values(record, ["001"])
    return result

def get_isbn(record):
    result = get_marc_values(record, ["010a"])
    return result

def get_ark(record):
    result = get_marc_values(record, ["033a"])
    return result

def get_alignement_bnf(metadata):
    result = False
    if 'ark:/12148' in metadata['ark']:
        result = True
    return result

def get_frbnf(record):
    result = ''
    data = get_marc_values(record, ["035a"])
    if 'FRBNF' in data:
        result = data
    return result

def get_refcom(record):
    result = get_marc_values(record, ["071ba"])
    return result

def get_ean(record):
    result = get_marc_values(record, ["073a"])
    return result

def get_support(record):
    result = get_marc_values(record, ["099t"])
    support_codes = {
            'AP': 'périodique - article',
            'CA': 'carte routière',
            'CR': 'cd-rom',
            'DC': 'disque compact',
            'DG': 'disque gomme-laque',
            'DV': 'disque microsillon',
            'IC': 'document iconographique',
            'JE': 'jeu',
            'K7': 'cassette audio',
            'LG': 'livre en gros caractères',
            'LI': 'livre',
            'LN': 'livre numérique',
            'LS': 'livre audio',
            'ML': 'méthode de langue',
            'MT': 'matériel',
            'PA': 'partition',
            'PE': 'périodique',
            'VD': 'dvd',
            'VI': 'vhs, umatic ou film'
        }
    if result in support_codes.keys():
        result = support_codes[result]
    return result

def get_global_title(record):
    result = get_marc_values(record, ["225a"])
    if result == '':
        result = get_marc_values(record, ["200ae"])
    return result

def get_title(record, global_titre=None):
    if global_titre:
        result = get_marc_values(record, ["200a"])
        result2 = get_marc_values(record, ["200ae"])
        if (result == global_titre) | (result2 == global_titre):
            result = get_marc_values(record, ["200i"])
    else:
        result = get_marc_values(record, ["200ae"])
    return result


def get_numero_tome(record):
    result = get_marc_values(record, ["200h"])
    if result == '':
        result = get_marc_values(record, ["461v"])
    return result

def get_responsability(record):
    result = get_marc_values(record, ["700ab", "710ab", "701ab", "711ab", "702ab", "712ab"])
    if result == '':
        get_marc_values(record, ["200f"])
    return result

def get_subject(record):
    result = get_marc_values(record, ["600abcdefghijklmnopqrstuvwxyz",
                                    "601abcdefghijklmnopqrstuvwxyz",
                                    "602abcdefghijklmnopqrstuvwxyz",
                                    "604abcdefghijklmnopqrstuvwxyz",
                                    "605abcdefghijklmnopqrstuvwxyz",
                                    "606abcdefghijklmnopqrstuvwxyz",
                                    "607abcdefghijklmnopqrstuvwxyz",
                                    "608abcdefghijklmnopqrstuvwxyz",
                                    "609abcdefghijklmnopqrstuvwxyz"])
    return result

def get_publication_date(record):
    result = get_marc_values(record, ["100a"])
    result = result[9:13]
    if result == '':
        result = get_marc_values(record, ["214d"])
    if result == '':
        result = get_marc_values(record, ["210d"])
    if result == '':
        result = get_marc_values(record, ["219d"])
    return result

def get_publisher(record):
    result = get_marc_values(record, ["214c"])
    if result == '':
        result = get_marc_values(record, ["210c"])
    if result == '':
        result = get_marc_values(record, ["219c"])
    return result

def get_publication_place(record):
    result = get_marc_values(record, ["214a"])
    if result == '':
        result = get_marc_values(record, ["210a"])
    if result == '':
        result = get_marc_values(record, ["219a"])
    return result

def get_nb_items(record):
    fields = record.get_fields('995')
    return len(fields)

### On exécute ces fonctions sur l'ensemble des notices du fichier
On obtient une liste de dictionnaires.

In [ ]:
metadatas = []
with open(marc_file, 'rb') as fh:
    reader = MARCReader(fh, to_unicode=True, force_utf8=True)
    for record in reader:
        metadata = {}
        metadata['type_notice'] = get_type_notice(record)
        metadata['niveau_bib'] = get_niveau_bib(record)
        metadata['record_id'] = get_record_id(record)
        metadata['isbn'] = get_isbn(record)
        metadata['ark'] = get_ark(record)
        metadata['frbnf'] = get_frbnf(record)
        metadata['refcom'] = get_refcom(record)
        metadata['ean'] = get_ean(record)
        metadata['support'] = get_support(record)
        metadata['publication_date'] =  get_publication_date(record)
        metadata['global_title'] = get_global_title(record)
        metadata['title'] = get_title(record, global_titre=metadata['global_title'])
        metadata['numero_tome'] = get_numero_tome(record)
        metadata['responsability'] = get_responsability(record)
        metadata['subject'] = get_subject(record)
        metadata['publisher'] = get_publisher(record)
        metadata['alignement_bnf'] = get_alignement_bnf(metadata)
        metadata['nb_items'] = get_nb_items(record)
        metadatas.append(metadata)

metadatas

### Export en excel
On transforme la liste de dictionnaires en Dataframe Pandas, qu'il est ensuite facile d'exporter en excel.

In [33]:
import pandas as pd

metadatas_df = pd.DataFrame.from_records(metadatas)
metadatas_df.to_excel("notices.xlsx", index=False)